In [1]:
import operator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV


In [2]:
diabetes = pd.read_csv('C:\\Users\\kasi\\Desktop\\datascience-master\\diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
Pregnancies                 768 non-null int64
Glucose                     768 non-null int64
BloodPressure               768 non-null int64
SkinThickness               768 non-null int64
Insulin                     768 non-null int64
BMI                         768 non-null float64
DiabetesPedigreeFunction    768 non-null float64
Age                         768 non-null int64
Outcome                     768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
diabetes.shape

(768, 9)

In [5]:
diabetes.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
diabetes.groupby('Outcome').size()

Outcome
0    500
1    268
dtype: int64

In [7]:
diabetes.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [8]:
feature_names = diabetes.columns[:8]
feature_names
X = diabetes[feature_names]
y = diabetes.Outcome


In [9]:
# Split your data into training and testing (80% / 20%)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=42,
    test_size=0.20
)

In [10]:
# I temporarily removed a few Glucose, BloodPressure and BMI rows with 0 values for better RFE result
diabetes_mod = diabetes[(diabetes.BloodPressure != 0) & (diabetes.BMI != 0) & (diabetes.Glucose != 0)]
diabetes_mod.shape

(724, 9)

In [12]:
log_reg = LogisticRegression(
    # Parameters chosen based on GridSearchCV result
    C=1,
    multi_class='ovr',
    penalty='l2',
    solver='newton-cg',
    random_state=42
)

In [13]:
log_reg.fit(X_train, y_train)
log_reg_predict = log_reg.predict(X_test)
log_reg_predict_proba = log_reg.predict_proba(X_test)[:, 1]

print('Accuracy of Logistic on training set: {:.2f}'.format(log_reg.score(X_train, y_train)))
print('Accuracy of Logistic  on test set: {:.2f}'.format(log_reg.score(X_test, y_test)))

#Model evaluation
print('Accuracy: {:.2f}%'.format(accuracy_score(y_test, log_reg_predict) * 100))
print('AUC: {:.2f}%'.format(roc_auc_score(y_test, log_reg_predict_proba) * 100))
print('Classification report:\n\n', classification_report(y_test, log_reg_predict))
print('Training set score: {:.2f}%'.format(log_reg.score(X_train, y_train) * 100))
print('Testing set score: {:.2f}%'.format(log_reg.score(X_test, y_test) * 100))

Accuracy of Logistic on training set: 0.77
Accuracy of Logistic  on test set: 0.75
Accuracy: 74.68%
AUC: 81.29%
Classification report:

              precision    recall  f1-score   support

          0       0.81      0.79      0.80        99
          1       0.64      0.67      0.65        55

avg / total       0.75      0.75      0.75       154

Training set score: 77.04%
Testing set score: 74.68%


In [19]:
pca = PCA(n_components=2)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

log_reg.fit(X_train, y_train)
log_reg_predict = log_reg.predict(X_test)
log_reg_predict_proba = log_reg.predict_proba(X_test)[:, 1]

print('Accuracy of Logistic with PCA on training set: {:.2f}'.format(log_reg.score(X_train, y_train)))
print('Accuracy of Logistic with PCA on test set: {:.2f}'.format(log_reg.score(X_test, y_test)))

#Model evaluation
print('Accuracy: {:.2f}%'.format(accuracy_score(y_test, log_reg_predict) * 100))
print('AUC: {:.2f}%'.format(roc_auc_score(y_test, log_reg_predict_proba) * 100))
print('Classification report:\n\n', classification_report(y_test, log_reg_predict))
print('Training set score: {:.2f}%'.format(log_reg.score(X_train, y_train) * 100))
print('Testing set score: {:.2f}%'.format(log_reg.score(X_test, y_test) * 100))


Accuracy of Logistic with PCA on training set: 0.73
Accuracy of Logistic with PCA on test set: 0.64
Accuracy: 64.29%
AUC: 21.84%
Classification report:

              precision    recall  f1-score   support

          0       0.64      1.00      0.78        99
          1       0.00      0.00      0.00        55

avg / total       0.41      0.64      0.50       154

Training set score: 73.29%
Testing set score: 64.29%


C:\Users\kasi\Anaconda3\envs\TensorFlow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
from sklearn.decomposition import FastICA 
ICA = FastICA(n_components=2, random_state=12) 
X=ICA.fit_transform(X_train)
X_test = ICA.transform(X_test)

log_reg.fit(X_train, y_train)
log_reg_predict = log_reg.predict(X_test)
log_reg_predict_proba = log_reg.predict_proba(X_test)[:, 1]
print('Accuracy of Logistic with ICA on training set: {:.2f}'.format(log_reg.score(X_train, y_train)))
print('Accuracy of Logistic with ICA on test set: {:.2f}'.format(log_reg.score(X_test, y_test)))

#Model evaluation
print('Accuracy: {:.2f}%'.format(accuracy_score(y_test, log_reg_predict) * 100))
print('AUC: {:.2f}%'.format(roc_auc_score(y_test, log_reg_predict_proba) * 100))
print('Classification report:\n\n', classification_report(y_test, log_reg_predict))
print('Training set score: {:.2f}%'.format(log_reg.score(X_train, y_train) * 100))
print('Testing set score: {:.2f}%'.format(log_reg.score(X_test, y_test) * 100))



Accuracy of Logistic with ICA on training set: 0.73
Accuracy of Logistic with ICA on test set: 0.64
Accuracy: 64.29%
AUC: 77.25%
Classification report:

              precision    recall  f1-score   support

          0       0.64      1.00      0.78        99
          1       0.00      0.00      0.00        55

avg / total       0.41      0.64      0.50       154

Training set score: 73.29%
Testing set score: 64.29%


C:\Users\kasi\Anaconda3\envs\TensorFlow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
